In [ ]:
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

#import locale
from datetime import datetime, timedelta
import time

### Development

In [2]:
driver = webdriver.Firefox()

In [3]:
driver.get('https://kiel-sailing-city.de/veranstaltungen/kalender')

In [4]:
# cookie rejection
try:
    # Wait for the button to be present and clickable
    decline_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.cm-btn.cm-btn-danger.cn-decline"))
    )
    # Click the button
    decline_button.click()
    #print("Cookies rejected.")
except Exception as e:
    print(f"An error occurred: {e}")

In [6]:
# extract aber ohne scrollen

try:
    # Corrected CSS selector with escaped square brackets
    elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.flex.flex-nowrap.bg-gray-200.min-h-\\[430px\\].flex-col"))
    )
    
    # Iterate over the elements and extract text
    for element in elements:
        # Extract the title
        title = element.find_element(By.CSS_SELECTOR, "h4.c-headline.c-rich-text.truncate").text
        
        # Extract the location
        location = element.find_element(By.CSS_SELECTOR, "span.text-base p").text
        
        # Extract the category tags
        categories = [span.text for span in element.find_elements(By.CSS_SELECTOR, "div.my-3.flex.flex-wrap span")]
        
        # Extract the time details
        time_details = element.find_element(By.CSS_SELECTOR, "div.text-xs").text

        # Extract the source link (href)
        source_link = element.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        
        # Print or store the extracted information
        print("Title:", title)
        print("Location:", location)
        print("Categories:", categories)
        print("Time Details:", time_details)
        print("Source Link:", source_link)
        print("-" * 40)
        
except Exception as e:
    print(f"An error occurred: {e}")


Title: Der Mensch dahinter
Location: Rathaus Kiel
Categories: ['Ausstellung', 'Kultur']
Time Details: Heute 08:00 - 18:00 Uhr
Endet in 3 Stunden
Source Link: https://kiel-sailing-city.de/e/der-mensch-dahinter/o-e443a00d-a39a-4556-bfc1-b89d3a36c3b4
----------------------------------------
Title: Weihnachtsmarkt im CITTI-Park
Location: CITTI-PARK
Categories: ['Gastronomie', 'Kinderprogramm', 'Konzert', 'Märkte', 'Regionale Spezialitäten', 'Shopping', 'Sport/Freizeit', 'Kultur']
Time Details: Heute 09:00 - 20:00 Uhr
Endet in 5 Stunden
Source Link: https://kiel-sailing-city.de/e/weihnachtsmarkt-im-citti-park/o-2ec8df79-2ccb-4f6e-aed4-e2707bc38428
----------------------------------------
Title: Mit Herodot auf Reisen
Location: Universitätsbibliothek Kiel
Categories: ['Ausstellung', 'Kultur', 'Wissenschaft']
Time Details: Heute 09:00 - 22:00 Uhr
Endet in 7 Stunden
Source Link: https://serviceaward-kiel.de/mit-herodot-auf-reisen/o-baf0c1fd-867b-43d1-8762-ef9132d28536
-------------------------

In [37]:
# Get the current date
current_date = datetime.now()

# Format the date as DD.MM.YYYY
formatted_date = current_date.strftime("%d.%m.%Y")

print(formatted_date)

26.09.2024


In [38]:
def generate_new_day_string(days_in_advance=10):
    # Get the current date
    current_date = datetime.now()
    today = current_date.strftime("%d.%m.%Y")

    # Calculate the date 10 days from today
    future_date = current_date + timedelta(days=days_in_advance)
    # Format the future date as DD.MM.YYYY
    formatted_future_date = future_date.strftime("%d.%m.%Y")

    new_date_string = today + " - " + formatted_future_date
    return new_date_string


In [62]:
generate_new_day_string(3)

'26.09.2024 - 29.09.2024'

In [71]:
#new_date_string = "26.09.2024 - 30.09.2024"
new_date_string = generate_new_day_string(3)

In [72]:
# zeitraum auswählen zb 10 tage im voraus

try:
    # Wait for the input field to be present
    input_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='DD.MM.YYYY  -  DD.MM.YYYY']"))
    )
    
    # Click the input field to focus it
    input_field.click()
    
    # Clear the existing value
    input_field.clear()

    # input days
    input_field.send_keys(new_date_string)

except Exception as e:
    print(f"An error occurred: {e}")

In [31]:
#extract mit scrollen

def extract_elements():
    elements = driver.find_elements(By.CSS_SELECTOR, "li.flex.flex-nowrap.bg-gray-200.min-h-\\[430px\\].flex-col")
    extracted_data = []
    
    for element in elements:
        try:
            # Extract the title
            title = element.find_element(By.CSS_SELECTOR, "h4.c-headline.c-rich-text.truncate").text
            
            # Extract the location
            location = element.find_element(By.CSS_SELECTOR, "span.text-base p").text
            
            # Extract the category tags
            categories = [span.text for span in element.find_elements(By.CSS_SELECTOR, "div.my-3.flex.flex-wrap span")]
            
            # Extract the time details
            time_details = element.find_element(By.CSS_SELECTOR, "div.text-xs").text
            
            # Store the extracted information
            extracted_data.append({
                "Title": title,
                "Location": location,
                "Categories": categories,
                "Time Details": time_details
            })
        except Exception as e:
            print(f"An error occurred while extracting element data: {e}")
    
    return extracted_data

# Scroll and extract data
all_data = []
previous_length = 0

try:
    while True:
        # Extract elements and append to all_data
        new_data = extract_elements()
        all_data.extend(new_data)
        
        # Check if new elements were found
        if len(all_data) == previous_length: #TODO löst bisher nicht aus
            break
        
        previous_length = len(all_data)
        
        # Scroll down the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait for 5 seconds
        time.sleep(3)
        
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Print extracted data
    for data in all_data:
        print("Title:", data["Title"])
        print("Location:", data["Location"])
        print("Categories:", data["Categories"])
        print("Time Details:", data["Time Details"])
        print("-" * 40)
    
    # Close the WebDriver
    #driver.quit()

Title: J. F. Kennedy - Sein Leben in Bildern
Location: Rathaus Kiel
Categories: ['Ausstellung', 'Kultur']
Time Details: Heute 08:00 - 18:00 Uhr
Endet in 6 Stunden
----------------------------------------
Title: 25. KUNSTMEILE KIEL
Location: CITTI-PARK
Categories: ['Ausstellung', 'Kultur']
Time Details: Heute 08:00 - 20:00 Uhr
Endet in 8 Stunden
----------------------------------------
Title: Der Gewölbekeller. Kostbarkeiten aus dem Alten Kiel
Location: Stadtmuseum Warleberger Hof
Categories: ['Ausstellung', 'Kultur', 'Kultursommer']
Time Details: Heute 10:00 - 18:00 Uhr
Endet in 6 Stunden
----------------------------------------
Title: „Juhu, 50 Jahre Spiellinie!“ - Ausstellung zum 50-jährigen Jubiläum
Location: vhs-Kunstschule im Neuen Rathaus
Categories: ['Ausstellung', 'Kultursommer']
Time Details: Heute 10:00 - 18:00 Uhr
Endet in 6 Stunden
----------------------------------------
Title: Kaufrausch. Konsum und Warenwelten der 1980er Jahre
Location: Stadtmuseum Warleberger Hof
Catego

KeyboardInterrupt: 

In [73]:
# extract mit scrollen und abbruch 
def extract_elements():
    elements = driver.find_elements(By.CSS_SELECTOR, "li.flex.flex-nowrap.bg-gray-200.min-h-\\[430px\\].flex-col")
    extracted_data = []
    
    for element in elements:
        try:
            # Extract the title
            title = element.find_element(By.CSS_SELECTOR, "h4.c-headline.c-rich-text.truncate").text
            
            # Extract the location
            location = element.find_element(By.CSS_SELECTOR, "span.text-base p").text
            
            # Extract the category tags
            categories = [span.text for span in element.find_elements(By.CSS_SELECTOR, "div.my-3.flex.flex-wrap span")]
            
            # Extract the time details
            time_details = element.find_element(By.CSS_SELECTOR, "div.text-xs").text
            
            # Store the extracted information
            extracted_data.append({
                "Title": title,
                "Location": location,
                "Categories": categories,
                "Time Details": time_details
            })
        except Exception as e:
            print(f"An error occurred while extracting element data: {e}")
    
    return extracted_data

# Scroll and extract data
all_data = []
previous_length = 0
previous_scroll_position = 0

try:
    while True:
        # Extract elements and append to all_data
        #new_data = extract_elements()
        #print(new_data)
        # If no new data is found, break the loop
        #if not new_data:
        #    print("No new data found. Exiting loop.")
        #    break
        
        #all_data.extend(new_data)
        
        # Check if new elements were found
        #if len(all_data) == previous_length:
        #    print("No new elements found. Exiting loop.")
        #    break
        
        #previous_length = len(all_data)
        
        # Scroll down the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait for 5 seconds
        time.sleep(3)

        # Get the current scroll position
        current_scroll_position = driver.execute_script("return window.pageYOffset;")
        
        # Check if the scroll position has changed
        if current_scroll_position == previous_scroll_position:
            # Scroll down the page
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            print("Reached the bottom of the page. Exiting loop.")
            all_data = extract_elements()
            break
        
        previous_scroll_position = current_scroll_position
        
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Print extracted data
    for data in all_data:
        print("Title:", data["Title"])
        print("Location:", data["Location"])
        print("Categories:", data["Categories"])
        print("Time Details:", data["Time Details"])
        print("-" * 40)


Reached the bottom of the page. Exiting loop.
Title: 25. KUNSTMEILE KIEL
Location: CITTI-PARK
Categories: ['Ausstellung', 'Kultur']
Time Details: Heute 08:00 - 20:00 Uhr
Endet in 6 Stunden
----------------------------------------
Title: J. F. Kennedy - Sein Leben in Bildern
Location: Rathaus Kiel
Categories: ['Ausstellung', 'Kultur']
Time Details: Heute 08:00 - 18:00 Uhr
Endet in 4 Stunden
----------------------------------------
Title: Der Gewölbekeller. Kostbarkeiten aus dem Alten Kiel
Location: Stadtmuseum Warleberger Hof
Categories: ['Ausstellung', 'Kultur', 'Kultursommer']
Time Details: Heute 10:00 - 18:00 Uhr
Endet in 4 Stunden
----------------------------------------
Title: Große Nord-Ostsee-Kanalfahrt
Location: Bahnhofskai
Categories: ['Kultur', 'Führung/Besichtigung', 'Kultursommer']
Time Details: Heute 10:00 - 17:30 Uhr
Endet in 4 Stunden
----------------------------------------
Title: Marine, Werften, Segelsport – die Geschichte der Hafenstadt Kiel
Location: Schifffahrtsmuse

In [82]:
#driver.quit()
driver.close()

In [74]:
all_data

[{'Title': '25. KUNSTMEILE KIEL',
  'Location': 'CITTI-PARK',
  'Categories': ['Ausstellung', 'Kultur'],
  'Time Details': 'Heute 08:00 - 20:00 Uhr\nEndet in 6 Stunden'},
 {'Title': 'J. F. Kennedy - Sein Leben in Bildern',
  'Location': 'Rathaus Kiel',
  'Categories': ['Ausstellung', 'Kultur'],
  'Time Details': 'Heute 08:00 - 18:00 Uhr\nEndet in 4 Stunden'},
 {'Title': 'Der Gewölbekeller. Kostbarkeiten aus dem Alten Kiel',
  'Location': 'Stadtmuseum Warleberger Hof',
  'Categories': ['Ausstellung', 'Kultur', 'Kultursommer'],
  'Time Details': 'Heute 10:00 - 18:00 Uhr\nEndet in 4 Stunden'},
 {'Title': 'Große Nord-Ostsee-Kanalfahrt',
  'Location': 'Bahnhofskai',
  'Categories': ['Kultur', 'Führung/Besichtigung', 'Kultursommer'],
  'Time Details': 'Heute 10:00 - 17:30 Uhr\nEndet in 4 Stunden'},
 {'Title': 'Marine, Werften, Segelsport – die Geschichte der Hafenstadt Kiel',
  'Location': 'Schifffahrtsmuseum',
  'Categories': ['Ausstellung', 'Kultursommer', 'Kultur', 'Kieler-Woche'],
  'Tim

In [77]:
# Current date
current_date = datetime.now().strftime('%d.%m.%Y')
current_date

'26.09.2024'

In [78]:
# Function to parse "Time Details"
def parse_time_details(time_details):
    date_time = time_details.split('\n')[0]  # Get the part before the newline
    if 'Heute' in date_time:
        date_time = date_time.replace('Heute', current_date)
    date, time = date_time.split(' ', 1)
    return date, time

In [79]:
# Parse the "Time Details" and create new columns
for item in all_data:
    date, time = parse_time_details(item['Time Details'])
    item['Date'] = date
    item['Time'] = time

In [80]:
all_data

[{'Title': '25. KUNSTMEILE KIEL',
  'Location': 'CITTI-PARK',
  'Categories': ['Ausstellung', 'Kultur'],
  'Time Details': 'Heute 08:00 - 20:00 Uhr\nEndet in 6 Stunden',
  'Date': '26.09.2024',
  'Time': '08:00 - 20:00 Uhr'},
 {'Title': 'J. F. Kennedy - Sein Leben in Bildern',
  'Location': 'Rathaus Kiel',
  'Categories': ['Ausstellung', 'Kultur'],
  'Time Details': 'Heute 08:00 - 18:00 Uhr\nEndet in 4 Stunden',
  'Date': '26.09.2024',
  'Time': '08:00 - 18:00 Uhr'},
 {'Title': 'Der Gewölbekeller. Kostbarkeiten aus dem Alten Kiel',
  'Location': 'Stadtmuseum Warleberger Hof',
  'Categories': ['Ausstellung', 'Kultur', 'Kultursommer'],
  'Time Details': 'Heute 10:00 - 18:00 Uhr\nEndet in 4 Stunden',
  'Date': '26.09.2024',
  'Time': '10:00 - 18:00 Uhr'},
 {'Title': 'Große Nord-Ostsee-Kanalfahrt',
  'Location': 'Bahnhofskai',
  'Categories': ['Kultur', 'Führung/Besichtigung', 'Kultursommer'],
  'Time Details': 'Heute 10:00 - 17:30 Uhr\nEndet in 4 Stunden',
  'Date': '26.09.2024',
  'Time'

In [81]:
df = pd.DataFrame(all_data)
df = df.drop(columns=['Time Details'])
df

,Title,Location,Categories,Date,Time
0,25. KUNSTMEILE KIEL,CITTI-PARK,"[Ausstellung, Kultur]",26.09.2024,08:00 - 20:00 Uhr
1,J. F. Kennedy - Sein Leben in Bildern,Rathaus Kiel,"[Ausstellung, Kultur]",26.09.2024,08:00 - 18:00 Uhr
2,Der Gewölbekeller. Kostbarkeiten aus dem Alten...,Stadtmuseum Warleberger Hof,"[Ausstellung, Kultur, Kultursommer]",26.09.2024,10:00 - 18:00 Uhr
3,Große Nord-Ostsee-Kanalfahrt,Bahnhofskai,"[Kultur, Führung/Besichtigung, Kultursommer]",26.09.2024,10:00 - 17:30 Uhr
4,"Marine, Werften, Segelsport – die Geschichte d...",Schifffahrtsmuseum,"[Ausstellung, Kultursommer, Kultur, Kieler-Woche]",26.09.2024,10:00 - 18:00 Uhr
...,...,...,...,...,...
165,Hafenrundfahrt Kiel,Anleger Bahnhofsbrücke,"[Führung/Besichtigung, Kultur, Kultursommer]",29.09.2024,15:00 Uhr
166,Auf Wasserwegen | Skulpturen und Zeichnungen v...,Galerie Brennwald,[Ausstellung],29.09.2024,15:00 - 18:00 Uhr
167,Figurentheater Marc Schnittger zeigt: Peers Mo...,KulturForum in der Stadtgalerie,"[Kinderprogramm, Kultur, Theater]",29.09.2024,16:30 - 17:30 Uhr
168,Elvis lebt!,Theater Die Komödianten,"[Kultur, Theater]",29.09.2024,18:00 - 19:30 Uhr


### Function

In [7]:
def generate_new_day_string(days_in_advance=10):
    # Get the current date
    current_date = datetime.now()
    today = current_date.strftime("%d.%m.%Y")

    # Calculate the date 10 days from today
    future_date = current_date + timedelta(days=days_in_advance)
    # Format the future date as DD.MM.YYYY
    formatted_future_date = future_date.strftime("%d.%m.%Y")

    new_date_string = today + " - " + formatted_future_date
    return new_date_string

In [8]:
# extract 
def extract_elements(driver):
    elements = driver.find_elements(By.CSS_SELECTOR, "li.flex.flex-nowrap.bg-gray-200.min-h-\\[430px\\].flex-col")
    extracted_data = []
    
    for element in elements:
        try:
            # Extract the title
            title = element.find_element(By.CSS_SELECTOR, "h4.c-headline.c-rich-text.truncate").text
            
            # Extract the location
            location = element.find_element(By.CSS_SELECTOR, "span.text-base p").text
            
            # Extract the category tags
            categories = [span.text for span in element.find_elements(By.CSS_SELECTOR, "div.my-3.flex.flex-wrap span")]
            
            # Extract the time details
            time_details = element.find_element(By.CSS_SELECTOR, "div.text-xs").text

            # Extract source link
            source_link = element.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            
            # Store the extracted information
            extracted_data.append({
                "Title": title,
                "Location": location,
                "Categories": categories,
                "Time Details": time_details,
                "Source": source_link
            })
        except Exception as e:
            print(f"An error occurred while extracting element data: {e}")
    
    return extracted_data

In [9]:
# Function to parse "Time Details"
def parse_time_details(time_details, current_date):
    date_time = time_details.split('\n')[0]  # Get the part before the newline
    if 'Heute' in date_time:
        date_time = date_time.replace('Heute', current_date)
    date, time_clock = date_time.split(' ', 1)
    return date, time_clock

In [10]:
def scrape_kiel_sailing_city(days_in_advance=10):
    
    driver = webdriver.Firefox()
    driver.get('https://kiel-sailing-city.de/veranstaltungen/kalender')
    time.sleep(5)

    # cookie rejection
    try:
        # Wait for the button to be present and clickable
        decline_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.cm-btn.cm-btn-danger.cn-decline"))
        )
        # Click the button
        decline_button.click()
        #print("Cookies rejected.")
    except Exception as e:
        print(f"An error occurred: {e}")

    new_date_string = generate_new_day_string(days_in_advance=days_in_advance)
    print(new_date_string)
    time.sleep(2)

    # zeitraum auswählen zb 10 tage im voraus
    try:
        # Wait for the input field to be present
        input_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[placeholder='DD.MM.YYYY  -  DD.MM.YYYY']"))
        )
        
        # Click the input field to focus it
        input_field.click()
        
        # Clear the existing value
        input_field.clear()

        # input days
        input_field.send_keys(new_date_string)

    except Exception as e:
        print(f"An error occurred: {e}")

    time.sleep(2)
    previous_scroll_position = 0
    all_data = []

    try:
        while True:
    
            # Scroll down the page
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Wait for 5 seconds
            time.sleep(3)

            # Get the current scroll position
            current_scroll_position = driver.execute_script("return window.pageYOffset;")
            
            # Check if the scroll position has changed
            if current_scroll_position == previous_scroll_position:
                # Scroll down the page
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                #print("Reached the bottom of the page. Exiting loop.")
                all_data = extract_elements(driver=driver)
                break
            
            previous_scroll_position = current_scroll_position
            
    except Exception as e:
        print(f"An error occurred: {e}")

    #driver.close()
    # Current date
    current_date = datetime.now().strftime('%d.%m.%Y')

    # Parse the "Time Details" and create new columns
    for item in all_data:
        date, time_clock = parse_time_details(item['Time Details'], current_date=current_date)
        item['Date'] = date
        item['Time'] = time_clock
    
    df = pd.DataFrame(all_data)
    df = df.drop(columns=['Time Details'])

    driver.close()

    return df


In [41]:
scraped_events_kiel = scrape_kiel_sailing_city(1)
scraped_events_kiel

28.11.2024 - 29.11.2024


,Title,Location,Categories,Source,Date,Time
0,Der Mensch dahinter,Rathaus Kiel,"[Ausstellung, Kultur]",https://kiel-sailing-city.de/e/der-mensch-dahi...,28.11.2024,08:00 - 18:00 Uhr
1,Mit Herodot auf Reisen,Universitätsbibliothek Kiel,"[Ausstellung, Kultur, Wissenschaft]",https://serviceaward-kiel.de/mit-herodot-auf-r...,28.11.2024,09:00 - 22:00 Uhr
2,Weihnachtsmarkt im CITTI-Park,CITTI-PARK,"[Gastronomie, Kinderprogramm, Konzert, Märkte,...",https://kiel-sailing-city.de/e/weihnachtsmarkt...,28.11.2024,09:00 - 20:00 Uhr
3,Weihnachten im Sophienhof,Sophienhof,"[Kinderprogramm, Shopping, Kultur, Märkte]",https://kiel-sailing-city.de/e/weihnachten-im-...,28.11.2024,10:00 - 20:00 Uhr
4,Kiel festhalten. Stadtansichten von Gretel Rie...,Stadtmuseum Warleberger Hof,"[Ausstellung, Kultur]",https://kiel-sailing-city.de/e/kiel-festhalten...,28.11.2024,10:00 - 18:00 Uhr
5,Kaufrausch. Konsum und Warenwelten der 1980er ...,Stadtmuseum Warleberger Hof,"[Ausstellung, Kultur, Kultursommer]",https://kiel-sailing-city.de/e/kaufrausch-kons...,28.11.2024,10:00 - 18:00 Uhr
6,Wohltemperiert. Für klimagerechte Architektur!,Freilichtmuseum Molfsee,"[Ausstellung, Kultur]",https://serviceaward-kiel.de/wohltemperiert-fu...,28.11.2024,10:00 - 16:00 Uhr
7,Der Gewölbekeller. Kostbarkeiten aus dem Alten...,Stadtmuseum Warleberger Hof,"[Ausstellung, Kultur, Kultursommer]",https://kiel-sailing-city.de/e/der-gewoelbekel...,28.11.2024,10:00 - 18:00 Uhr
8,"Riesenrad am Bahnhof ""La Noria""",Kiel Hauptbahnhof,"[Kultur, Sport/Freizeit]",https://kiel-sailing-city.de/e/riesenrad-am-ba...,28.11.2024,11:00 - 21:00 Uhr
9,Kunstraum Sconeberg mit ganzjährigem Ausstellu...,Kunstraum Sconeberg,"[Ausstellung, Kultur]",https://kiel-sailing-city.de/e/kunstraum-scone...,28.11.2024,11:00 - 18:00 Uhr


In [22]:
#df_raw = scraped_events_kiel
df_raw.head()

,Subject,Location,Categories,Description,Start_date,End_date,Start_time,End_time
0,Der Mensch dahinter,Rathaus Kiel,"[Ausstellung, Kultur]",https://kiel-sailing-city.de/e/der-mensch-dahi...,28.11.2024,28.11.2024,08:00,18:00
1,Weihnachtsmarkt im CITTI-Park,CITTI-PARK,"[Gastronomie, Kinderprogramm, Konzert, Märkte,...",https://kiel-sailing-city.de/e/weihnachtsmarkt...,28.11.2024,28.11.2024,09:00,20:00
2,Mit Herodot auf Reisen,Universitätsbibliothek Kiel,"[Ausstellung, Kultur, Wissenschaft]",https://serviceaward-kiel.de/mit-herodot-auf-r...,28.11.2024,28.11.2024,09:00,22:00
3,Wohltemperiert. Für klimagerechte Architektur!,Freilichtmuseum Molfsee,"[Ausstellung, Kultur]",https://serviceaward-kiel.de/wohltemperiert-fu...,28.11.2024,28.11.2024,10:00,16:00
4,Kaufrausch. Konsum und Warenwelten der 1980er ...,Stadtmuseum Warleberger Hof,"[Ausstellung, Kultur, Kultursommer]",https://kiel-sailing-city.de/e/kaufrausch-kons...,28.11.2024,28.11.2024,10:00,18:00


In [14]:
df_raw.rename(columns={'Title': 'Subject'}, inplace=True)

In [16]:
df_raw.rename(columns={'Date': 'Start_date'}, inplace=True)
df_raw["End_date"] = df_raw["Start_date"]

In [19]:
df_raw[['Start_time', 'End_time']] = df_raw['Time'].apply(preprocess_time)
df_raw.drop(columns=['Time'], inplace=True)

In [ ]:
df_raw.rename(columns={'Source': 'Description'}, inplace=True) 

In [ ]:
music = ['Nachtleben', 'Konzert', 'konzert', 'Musik', 'musik', 'Party', 'party', 'Tanz', 'tanz', 'Festival', 'festival', 'Musical', 'musical', 'Jazz', 'jazz', 'Blues', 'blues', 'Country', 'country', 'Folk', 'folk', 'Rock', 'rock', 'Pop', 'pop', 'Klassik', 'klassik', 'Gospel', 'gospel', 'Chöre', 'chöre']

In [32]:
df_raw['Music_label'] = df_raw['Categories'].apply(check_music)

In [34]:
df_raw.rename(columns={'Categories': 'Category'}, inplace=True) 

In [35]:
df_raw["City"] = "Kiel"

In [38]:
df_prep = df_raw[['Subject','Start_date', 'End_date', 'Start_time', 'End_time', 'Location', 'City', 'Description', 'Category', 'Music_label']]
df_prep

,Subject,Start_date,End_date,Start_time,End_time,Location,City,Description,Category,Music_label
0,Der Mensch dahinter,28.11.2024,28.11.2024,08:00,18:00,Rathaus Kiel,Kiel,https://kiel-sailing-city.de/e/der-mensch-dahi...,"[Ausstellung, Kultur]",no music
1,Weihnachtsmarkt im CITTI-Park,28.11.2024,28.11.2024,09:00,20:00,CITTI-PARK,Kiel,https://kiel-sailing-city.de/e/weihnachtsmarkt...,"[Gastronomie, Kinderprogramm, Konzert, Märkte,...",music
2,Mit Herodot auf Reisen,28.11.2024,28.11.2024,09:00,22:00,Universitätsbibliothek Kiel,Kiel,https://serviceaward-kiel.de/mit-herodot-auf-r...,"[Ausstellung, Kultur, Wissenschaft]",no music
3,Wohltemperiert. Für klimagerechte Architektur!,28.11.2024,28.11.2024,10:00,16:00,Freilichtmuseum Molfsee,Kiel,https://serviceaward-kiel.de/wohltemperiert-fu...,"[Ausstellung, Kultur]",no music
4,Kaufrausch. Konsum und Warenwelten der 1980er ...,28.11.2024,28.11.2024,10:00,18:00,Stadtmuseum Warleberger Hof,Kiel,https://kiel-sailing-city.de/e/kaufrausch-kons...,"[Ausstellung, Kultur, Kultursommer]",no music
5,Der Gewölbekeller. Kostbarkeiten aus dem Alten...,28.11.2024,28.11.2024,10:00,18:00,Stadtmuseum Warleberger Hof,Kiel,https://kiel-sailing-city.de/e/der-gewoelbekel...,"[Ausstellung, Kultur, Kultursommer]",no music
6,Weihnachten im Sophienhof,28.11.2024,28.11.2024,10:00,20:00,Sophienhof,Kiel,https://kiel-sailing-city.de/e/weihnachten-im-...,"[Kinderprogramm, Shopping, Kultur, Märkte]",no music
7,Kiel festhalten. Stadtansichten von Gretel Rie...,28.11.2024,28.11.2024,10:00,18:00,Stadtmuseum Warleberger Hof,Kiel,https://kiel-sailing-city.de/e/kiel-festhalten...,"[Ausstellung, Kultur]",no music
8,Weihnachtsmarkt - Asmus-Bremer-Platz,28.11.2024,28.11.2024,11:00,21:00,Asmus-Bremer-Platz,Kiel,https://kiel-sailing-city.de/e/kieler-weihnach...,"[Gastronomie, Kinderprogramm, Regionale Spezia...",no music
9,"Riesenrad am Bahnhof ""La Noria""",28.11.2024,28.11.2024,11:00,21:00,Kiel Hauptbahnhof,Kiel,https://kiel-sailing-city.de/e/riesenrad-am-ba...,"[Kultur, Sport/Freizeit]",no music


In [31]:
def check_music(category_list):
    for category in category_list:
        for word in music:
            if word in category:
                return 'music'
    return 'no music'

In [18]:
# Helper functions and elements

def preprocess_time(time_str):
    if '-' in time_str:
        start_time, end_time = time_str.split(' - ')
        end_time = end_time.replace(' Uhr', '')
    else:
        start_time = time_str.replace(' Uhr', '')
        end_time = 'N/A'
    return pd.Series([start_time, end_time])

In [39]:
# Preprocessing function

def preprocess_kiel_sailing_city(df_raw):

    df_raw.rename(columns={'Title': 'Subject'}, inplace=True)

    df_raw.rename(columns={'Date': 'Start_date'}, inplace=True)
    df_raw["End_date"] = df_raw["Start_date"]

    df_raw[['Start_time', 'End_time']] = df_raw['Time'].apply(preprocess_time)
    df_raw.drop(columns=['Time'], inplace=True)

    df_raw.rename(columns={'Source': 'Description'}, inplace=True)    

    df_raw['Music_label'] = df_raw['Categories'].apply(check_music)

    df_raw.rename(columns={'Categories': 'Category'}, inplace=True) 

    df_raw["City"] = "Kiel"

    df_prep = df_raw[['Subject','Start_date', 'End_date', 'Start_time', 'End_time', 'Location', 'City', 'Description', 'Category', 'Music_label']]
    return df_prep

In [42]:
preprocess_kiel_sailing_city(scraped_events_kiel)

,Subject,Start_date,End_date,Start_time,End_time,Location,City,Description,Category,Music_label
0,Der Mensch dahinter,28.11.2024,28.11.2024,08:00,18:00,Rathaus Kiel,Kiel,https://kiel-sailing-city.de/e/der-mensch-dahi...,"[Ausstellung, Kultur]",no music
1,Mit Herodot auf Reisen,28.11.2024,28.11.2024,09:00,22:00,Universitätsbibliothek Kiel,Kiel,https://serviceaward-kiel.de/mit-herodot-auf-r...,"[Ausstellung, Kultur, Wissenschaft]",no music
2,Weihnachtsmarkt im CITTI-Park,28.11.2024,28.11.2024,09:00,20:00,CITTI-PARK,Kiel,https://kiel-sailing-city.de/e/weihnachtsmarkt...,"[Gastronomie, Kinderprogramm, Konzert, Märkte,...",music
3,Weihnachten im Sophienhof,28.11.2024,28.11.2024,10:00,20:00,Sophienhof,Kiel,https://kiel-sailing-city.de/e/weihnachten-im-...,"[Kinderprogramm, Shopping, Kultur, Märkte]",no music
4,Kiel festhalten. Stadtansichten von Gretel Rie...,28.11.2024,28.11.2024,10:00,18:00,Stadtmuseum Warleberger Hof,Kiel,https://kiel-sailing-city.de/e/kiel-festhalten...,"[Ausstellung, Kultur]",no music
5,Kaufrausch. Konsum und Warenwelten der 1980er ...,28.11.2024,28.11.2024,10:00,18:00,Stadtmuseum Warleberger Hof,Kiel,https://kiel-sailing-city.de/e/kaufrausch-kons...,"[Ausstellung, Kultur, Kultursommer]",no music
6,Wohltemperiert. Für klimagerechte Architektur!,28.11.2024,28.11.2024,10:00,16:00,Freilichtmuseum Molfsee,Kiel,https://serviceaward-kiel.de/wohltemperiert-fu...,"[Ausstellung, Kultur]",no music
7,Der Gewölbekeller. Kostbarkeiten aus dem Alten...,28.11.2024,28.11.2024,10:00,18:00,Stadtmuseum Warleberger Hof,Kiel,https://kiel-sailing-city.de/e/der-gewoelbekel...,"[Ausstellung, Kultur, Kultursommer]",no music
8,"Riesenrad am Bahnhof ""La Noria""",28.11.2024,28.11.2024,11:00,21:00,Kiel Hauptbahnhof,Kiel,https://kiel-sailing-city.de/e/riesenrad-am-ba...,"[Kultur, Sport/Freizeit]",no music
9,Kunstraum Sconeberg mit ganzjährigem Ausstellu...,28.11.2024,28.11.2024,11:00,18:00,Kunstraum Sconeberg,Kiel,https://kiel-sailing-city.de/e/kunstraum-scone...,"[Ausstellung, Kultur]",no music


In [7]:
scraped_events_kiel.to_csv("events_kiel.csv")